In [1]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline

# Prep
## Load

In [2]:
df = pd.read_csv('housing-Copy1.csv')

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Checkout out the head.

In [3]:
num = [f for f in df.columns if df.dtypes[f] != 'object']
cat = [f for f in df.columns if df.dtypes[f] == 'object']
num.remove('median_house_value')
target = 'median_house_value' # Target. What we want to predict

## Preview
20640 rows with 10 variables. 8 numeric and 1 categorical. *median_house_value* is what we are trying to predict.
- Numeric: 'longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'
- Categorical: 'ocean_proximity'
- Target: 'median_house_value'

In [4]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


## Target
Understand what you are trying to predict

In [5]:
df[target].hist(bins=100);
plt.savefig('target-housing.png')
plt.close();

![](target-housing.png)

In [6]:
df[target].describe()

count     20640.000000
mean     206855.816909
std      115395.615874
min       14999.000000
25%      119600.000000
50%      179700.000000
75%      264725.000000
max      500001.000000
Name: median_house_value, dtype: float64

The **median_house_value** is our target and there is a cap in the target at 500,000.

## Create A Test Set
When splitting data, check first if there are similar groups you want to stratify to get an even random split.

In [7]:
df['income_cat'] = np.ceil(df[target] / 1.5)
df['income_cat'].where(df['income_cat'] < 5, 5.0, inplace=True)
num = num + ['income_cat']

Group median income according to 5 categories

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['income_cat']):
    train = df.loc[train_index]
    test = df.loc[test_index]

Using Startified Split

In [9]:
df.income_cat.value_counts() / len(df)

5.0    1.0
Name: income_cat, dtype: float64

The _Test Set_ generated using stratified sampling has income category proportions almost identical to those in the full dataset, whereas the _Test Set_ generated using purely random sampling is quite skewed.

You now have the Train dataset. From this point forward, all cleaning you do must be only done on Train. Test should not be touched because of it lead to overfitting when we come to evaluation.

# Explore Data
Start with checking correlations of numeric variables with the target data. Then make explore the data with many graphs but only show the top 3 graphs. Pairplot, scatterplot and a wildcard graph. Use only the train data in exploring.

## Correlations

In [10]:
train.corr()[target].sort_values(ascending=False).reset_index()

,index,median_house_value
0,median_house_value,1.000000
1,median_income,0.689752
2,total_rooms,0.136167
3,housing_median_age,0.107050
4,households,0.068976
5,total_bedrooms,0.052690
6,population,-0.024618
7,longitude,-0.041830
8,latitude,-0.148438
9,income_cat,NaN


Median_income has a strong relationship with the median_house_value

In [11]:
matrix = train[['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value']].corr()

np.fill_diagonal(matrix.values, 0)
sns.heatmap(matrix, vmin=0, vmax=1
            , cmap='Blues', annot=True, linecolor='lightgrey', linewidths=.1, cbar=False);
plt.savefig('matrix-calhousprices.png')
plt.close();

![](matrix-calhousprices.png)

There are definitely some strong relationships of features within each other, which is worth considering for a feature engineering.

## 3 Graphs

In [12]:
sns.pairplot(train[[target] + num[2:-1]]);
plt.savefig('housing-pairplot.png')
plt.close();

![](housing-pairplot.png)
Pairplot shows that there are features that correlates with other features. We can possibly combine them to have a much stronger feature that correlates with our target.

In [13]:
sns.regplot(x='median_income', y=target, data=train
            , scatter_kws={'alpha':0.5} , line_kws={"color": "orange"});
plt.savefig('median_income.png')
plt.close();

![](median_income.png)
median income is the most correlated feature for our target. There are lines at 500k where we see the cap but there are also lines between 450k and 350k, which needs to be address before we fit our model.

In [14]:
## Using Seaborn
# sns.relplot(x='longitude', y='latitude', data=train, hue='median_house_value'
#             , palette=plt.get_cmap('jet')
#             , size=train['population']/100, sizes=(50,500), alpha=0.3 );
## Using MatplotLib
train.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,
             s=train['population']/100, label='population', c=target, 
             cmap=plt.get_cmap('jet'), colorbar=True);
plt.savefig('california-heatmap.png')
plt.close();

![](california-heatmap.png)
Housing prices are very much related to the location and the population density. Clustering algorithm to detect main clusters and add new features that measure the proximity to the cluster centers.

# Data Cleaning
After doing an initial exploration, you must clean the dataset but start doing it only on **train**. As you clean it, you will build a pipeline of functions that you will use to clean the test set and any future dataset.

## Missing Data
Assess each numerical values for missing data and use the imputer to replace them with the median value.

In [15]:
mis = train.isnull().sum()
mis = mis[mis>0]
pd.DataFrame({
    'count': mis,
    'proportion': mis / len(train)
}).sort_values(by='proportion', ascending=False).style.bar()

,count,proportion
total_bedrooms,207,0.012536


In [16]:
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(strategy='median')
imp_median.fit(train[num])

pd.DataFrame(imp_median.transform(train[num]), columns=num).head() ## preview. can delete

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,income_cat
0,-118.01,33.79,23.0,2663.0,430.0,1499.0,403.0,5.7837,5.0
1,-117.84,33.73,20.0,2572.0,732.0,1534.0,669.0,2.4211,5.0
2,-117.60,33.87,15.0,7626.0,1570.0,3823.0,1415.0,3.4419,5.0
3,-117.64,34.09,34.0,2839.0,659.0,1822.0,631.0,3.0500,5.0
4,-122.18,37.77,51.0,2107.0,471.0,1173.0,438.0,3.2552,5.0


Seems to be a lot of writing code for something simple. But here, we are getting used to *initiate, fit, transform* for sklearn, which will pay off in the long run once we build our automated pipeline.

## Categorical Encoding
For Categorical values, we need to encode them to zeros and ones.

In [17]:
# train = train.join(pd.get_dummies(train['ocean_proximity']))

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit_transform(train[cat])

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0]])

When encoding, it seems strange that we don't care of the columne names and just have the values zeros and ones. This must have a purpose later on. Better keep moving.

## Feature Engineering
There will be some features that would have a weak relationship with the target but are strongly correlated with other weak features. Exploring the combination of these features by combining them together may result into a stronger feature that we can use to improve our predictive capability of the target.

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [19]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False) # initiate
housing_extra_attributes = attr_adder.transform(train.values) # transform

The more you automate these data preparation steps, the more combinations you can automatically try out, making it much more lifely that you will find a great combination.

## Feature Scaling
Total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15.


### MinMaxScaler Normalization
Transforms the numeric data between 0 and 1.

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # initiate
scaler.fit(train[num]) # fit
minmax = scaler.transform(train[num]) # transform
pd.DataFrame(minmax, columns=num).describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,income_cat
count,16512.000000,16512.000000,16512.000000,16512.000000,16305.000000,16512.000000,16512.000000,16512.000000,16512.0
mean,0.476581,0.327853,0.542308,0.066839,0.083031,0.049609,0.081745,0.232449,0.0
std,0.199256,0.226671,0.246869,0.055535,0.065198,0.038838,0.062566,0.130775,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.254980,0.147715,0.333333,0.036701,0.045934,0.027415,0.046045,0.142531,0.0
50%,0.583665,0.182253,0.549020,0.053945,0.067039,0.040615,0.067094,0.209304,0.0
75%,0.631474,0.549416,0.705882,0.079963,0.099783,0.060117,0.098874,0.292641,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


### Standardization
Uses standard deviation with mean as zero.

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[num])
pd.DataFrame(scaler.transform(train[num]), columns=num).describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,income_cat
count,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.630500e+04,1.651200e+04,1.651200e+04,1.651200e+04,16512.0
mean,-1.538505e-15,7.477868e-16,3.805634e-17,-6.044616e-17,1.101984e-16,-1.053004e-16,2.669995e-17,-1.972744e-16,0.0
std,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000031e+00,1.000030e+00,1.000030e+00,1.000030e+00,0.0
min,-2.391881e+00,-1.446426e+00,-2.196807e+00,-1.203579e+00,-1.273565e+00,-1.277386e+00,-1.306592e+00,-1.777519e+00,0.0
25%,-1.112179e+00,-7.947334e-01,-8.465243e-01,-5.427055e-01,-5.690089e-01,-5.714807e-01,-5.706202e-01,-6.875957e-01,0.0
50%,5.374377e-01,-6.423593e-01,2.718806e-02,-2.321909e-01,-2.452939e-01,-2.316002e-01,-2.341759e-01,-1.769856e-01,0.0
75%,7.773818e-01,9.774942e-01,6.626152e-01,2.363289e-01,2.569403e-01,2.705575e-01,2.737761e-01,4.602849e-01,0.0
max,2.626952e+00,2.965390e+00,1.854041e+00,1.680352e+01,1.406481e+01,2.447149e+01,1.467714e+01,5.869411e+00,0.0


You must only *fit* scalers to the **train** data. You must not fit it to the whole data set or to the test set because this will cause overfitting when we do evaluation.

## Transformation Pipeline
Ok. This is where you get everything together. Do everything step by step.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelBinarizer
from sklearn.impute import SimpleImputer

In [23]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
class NewLabelBinarizer(LabelBinarizer):
    def fit(self, X, y=None):
        return super(NewLabelBinarizer, self).fit(X)
    def transform(self, X, y=None):
        return super(NewLabelBinarizer, self).transform(X)
    def fit_transform(self, X, y=None):
        return super(NewLabelBinarizer, self).fit(X).transform(X)

In [24]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num))
    , ('imputer', SimpleImputer(strategy='median'))
    , ('attribs_adder', CombinedAttributesAdder())
    , ('std_scaler', StandardScaler())
])

In [25]:
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat))
    , ('label_binarizer', NewLabelBinarizer())
])

In [26]:
full_pipeline = FeatureUnion([
        ('num_pipeline', num_pipeline)
        , ('cat_pipeline', cat_pipeline)
])

In [27]:
housing_prepared = full_pipeline.fit_transform(train)

# Machine Learning
## Do 3 Models
### Linear Regression

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
housing_prepared = full_pipeline.transform(train) # does not have target
lin_reg = LinearRegression() # init
lin_reg.fit(housing_prepared, train[target]) # fit
lin_mse = mean_squared_error(train[target], lin_reg.predict(housing_prepared)) # predict evaluate
np.sqrt(lin_mse)

67682.88191615137

This is not ideal

### DecisionTree

In [29]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor() # init
tree_reg.fit(housing_prepared, train[target]) # fit
tree_mse = mean_squared_error(train[target], tree_reg.predict(housing_prepared))# predict evaluate
np.sqrt(tree_mse)

0.0

This is overfitting

### Random Forest

In [30]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor() # init
forest_reg.fit(housing_prepared, train[target]) # fit
forest_mse = mean_squared_error(train[target], forest_reg.predict(housing_prepared))# predict evaluate
np.sqrt(forest_mse)

18554.320843542097

Better model to use.

## Cross Validation

In [31]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, train[target],
                        scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(rmse_scores.mean())
print(rmse_scores.std())

[69895.40004018 74650.42123281 66643.32392688 73955.53947841
 72935.3583096  73998.02773444 70954.9654254  67272.85749167
 72323.08086617 69767.0237909 ]
71239.5998296458
2675.5494811881103


In [32]:
scores = cross_val_score(lin_reg, housing_prepared, train[target],
                        scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(rmse_scores.mean())
print(rmse_scores.std())

[67609.58112566 68923.20005577 65767.71443054 67107.22038658
 68147.70871523 67422.36125738 68453.48347856 68142.56585259
 69394.88530646 67993.11871622]
67896.18393249981
959.3909955206789


In [33]:
scores = cross_val_score(forest_reg, housing_prepared, train[target],
                        scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)
print(rmse_scores.mean())
print(rmse_scores.std())

[50454.03815292 51894.21972801 46083.74313118 51509.5596845
 49364.22583881 49775.06014245 50541.26883921 48624.91315601
 51029.80795205 49657.83949033]
49893.46761154707
1584.7014409322464


## Grid Search
- 3 n_estimators x 4 max_features = 12
- 2 n_estimators x 3 max_features = 6
- 5 folds x 18 combinations = 90 rounds of training

In [34]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(housing_prepared, train[target])

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             scoring='neg_mean_squared_error')

In [35]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

This shows the best parameter that made the best results

In [36]:
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30)

You can see the best estimator to be used. Here we only placed 1.

In [37]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

63417.14320214591 {'max_features': 2, 'n_estimators': 3}
55260.48333977465 {'max_features': 2, 'n_estimators': 10}
52687.984815181604 {'max_features': 2, 'n_estimators': 30}
59706.5284663878 {'max_features': 4, 'n_estimators': 3}
52604.76228598347 {'max_features': 4, 'n_estimators': 10}
50305.02368865285 {'max_features': 4, 'n_estimators': 30}
59454.616007564895 {'max_features': 6, 'n_estimators': 3}
52038.98129554137 {'max_features': 6, 'n_estimators': 10}
49973.86149209295 {'max_features': 6, 'n_estimators': 30}
58107.450149458586 {'max_features': 8, 'n_estimators': 3}
51963.201925216395 {'max_features': 8, 'n_estimators': 10}
50005.763386448205 {'max_features': 8, 'n_estimators': 30}
62739.7824434678 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54240.50153446846 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60492.753573103095 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52589.011592316034 {'bootstrap': False, 'max_features': 3, 'n_estimators

You can see all the parameters and their corresponding scores here.

In [38]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([7.43472322e-02, 6.86640223e-02, 4.10868045e-02, 1.87403763e-02,
       1.72745621e-02, 1.73445520e-02, 1.58706964e-02, 3.46996742e-01,
       0.00000000e+00, 6.00676357e-02, 1.07544998e-01, 5.79257032e-02,
       7.84005456e-03, 1.57861569e-01, 2.09144088e-04, 2.10354217e-03,
       6.12236612e-03])

In [39]:
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_one_hot_attribs = list(encoder.classes_)
attributes = num + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3469967423703149, 'median_income'),
 (0.1578615685435408, 'INLAND'),
 (0.10754499758288126, 'pop_per_hhold'),
 (0.07434723215332577, 'longitude'),
 (0.06866402228588957, 'latitude'),
 (0.06006763570055924, 'rooms_per_hhold'),
 (0.05792570315087823, 'bedrooms_per_room'),
 (0.04108680451213495, 'housing_median_age'),
 (0.01874037626303376, 'total_rooms'),
 (0.017344551986151074, 'population'),
 (0.017274562107080314, 'total_bedrooms'),
 (0.015870696402697726, 'households'),
 (0.007840054562743804, '<1H OCEAN'),
 (0.0061223661164628304, 'NEAR OCEAN'),
 (0.0021035421745461426, 'NEAR BAY'),
 (0.00020914408775957882, 'ISLAND'),
 (0.0, 'income_cat')]

You can see which features are more stronger. Consider removing some weak ones.

## Evaluate in Test Set

In [40]:
final_model = grid_search.best_estimator_
X_test = test.drop(target, axis=1)
Y_test = test[target]
X_test_prep = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prep)
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

50207.62225615481

# Conclusions